In [13]:
parameters_content = '''
#include <stdio.h>
#include "include/gemmini.h"

#define LEN(arr) ((int) (sizeof (arr) / sizeof (arr[0])))

'''
SYS_DIM=32
batch_size=32
input_dim=3036
param_file_name="parameters8.h"
test_file_name="test8.c"
if(batch_size%SYS_DIM !=0):
    print('please use a batch size of SYS_DIM')
    exit(1)   
#layers = [input_dim,2500,2000,1500,1000,500,10]
layers=[input_dim,4554,3036]
mlp='// batch size: ' + str(batch_size) +'\n'
mlp+='// before zeropad: '
for l in layers:
    mlp=mlp+str(l)+'x'
    
parameters_content = parameters_content+mlp[:-1]+'\n'
for i,l in enumerate(list(layers)):
    layers[i] = int((l+SYS_DIM-1)/SYS_DIM)*SYS_DIM
    
mlp='// after zeropad: '
for l in layers:
    mlp=mlp+str(l)+'x'
parameters_content = parameters_content+mlp[:-1]+'\n'

input_mat = 'elem_t input_mat[{0}][{1}] row_align(1)= '.format(batch_size,layers[0]) + '{0};\n'
parameters_content = parameters_content+input_mat;

matmuls = []
for i,(l1,l2) in enumerate(zip(layers[:-1],layers[1:])):
    matrix = 'elem_t weights'+str(i)+'['+str(l1)+']'+'['+str(l2)+'] row_align(1)= {0};\n'
    parameters_content = parameters_content+matrix
    inter_results = 'elem_t inter_results'+str(i)+'['+str(batch_size)+']'+'['+str(l2)+'] row_align(1)= {0};\n'
    parameters_content = parameters_content+inter_results
    if i==0:
        matmuls.append(('input_mat',batch_size,'weights'+str(i),l1,'inter_results'+str(i),l2))
    else:
        matmuls.append(('inter_results'+str(i-1),batch_size,'weights'+str(i),l1,'inter_results'+str(i),l2))

f = open(param_file_name, "w")
f.write(parameters_content)
f.close()

test_content= '''

#include <stdio.h>
#include <string.h>
#include <stdbool.h>
#include <sys/mman.h>
#include "include/gemmini.h"
#include ''' +'"'+param_file_name+'"'+ '''

#define verbose(layer_num,old_C,filter,C) printf("layer %d: operand %d %d filter %d %d result %d %d\\n", layer_num, LEN(old_C),LEN(old_C[0]),LEN(filter),LEN(filter[0]),LEN(C),LEN(C[0]));

static void tiled_matmul_compare(size_t DIM_I, size_t DIM_J, size_t DIM_K,
        elem_t A[DIM_I][DIM_K], elem_t B[DIM_K][DIM_J], acc_t D[DIM_I][DIM_J],
        elem_t C[DIM_I][DIM_J], int no_bias, int act, int shift, int relu6_shift,
        enum tiled_matmul_type_t tiled_matmul_type,
        bool compare, char * layer_name)
{
    if (compare)
        printf("%s: gemmini\\n", layer_name);
    tiled_matmul_option(DIM_I, DIM_J, DIM_K,
        A, B, D, C, no_bias, act, shift, relu6_shift,
        tiled_matmul_type);

    if (compare) {
        printf("%s: CPU\\n", layer_name);
        elem_t gold[DIM_I][DIM_J];
        tiled_matmul_option(DIM_I, DIM_J, DIM_K,
            A, B, D, gold, no_bias, act, shift, relu6_shift,
            CPU);

        printf("%s: comparing\\n", layer_name);
        for (size_t i = 0; i < DIM_I; i++) {
            for (size_t j = 0; j < DIM_J; j++) {
                if (C[i][j] != gold[i][j]) {
                    printf("Layer calculated incorrectly: %s\\n", layer_name);
                    exit(1);
                }
            }
        }
    }
}

int main (int argc, char * argv[]) {
#ifndef BAREMETAL
    if (mlockall(MCL_CURRENT | MCL_FUTURE) != 0) {
      perror("mlockall failed");
      exit(1);
    }
#endif

    matmul_flush(0);

    enum tiled_matmul_type_t tiled_matmul_type;
    if (argc < 2) {
        // printf("usage: %s matmul_option\\n  matmul_option may be 'os', 'ws', or cpu'\\n");
        // exit(0);
        tiled_matmul_type = OS;
    } else if (strcmp(argv[1], "cpu") == 0) {
        tiled_matmul_type = CPU;
    } else if (strcmp(argv[1], "os") == 0) {
        tiled_matmul_type = OS;
    } else if (strcmp(argv[1], "ws") == 0) {
        tiled_matmul_type = WS;
    }

    bool compare;
    if (argc < 3) {
        compare = false;
    } else if (strcmp(argv[2], "compare") == 0) {
        compare = true;
    } else {
        printf("Unknown command-line argument\\n");
        exit(1);
    }


'''
test_content =  test_content + '    uint64_t cycles['+str(len(layers)-1)+']={0};\n'
test_content =  test_content + '    uint64_t start,end;\n'
test_content =  test_content + '    start = read_cycles();'
for m,(A,a,weight,w,C,c) in enumerate(matmuls):
    test_content +='\n\n    /* matmul number: {0} */\n\n'.format(str(m))
    test_content +='    tiled_matmul_compare({0}, {1}, {2},    // dimensions\n'.format(a,c,w)
    test_content +='    {0}, {1}, NULL, {2},      // addresses\n'.format(A,weight,C)
    test_content +='    1, RELU, 0, 0,              // no_bias, act, shift, r6_shift\n'
    test_content +='    tiled_matmul_type, compare, "layer_'+str(m)+'");\n'
    test_content +='    // verbose({0},{1},{2},{3})\n'.format(m,A,weight,C)
    test_content +='    /* end of matmul number: {0} */\n\n'.format(str(m))
    test_content +='    end = read_cycles();\n'
    test_content +='    cycles['+str(m)+'] = end-start;\n'
    test_content +='    start = end;\n'

test_content +='''
    uint64_t overall_cycles = 0;
    for(int cyc = 0; cyc < '''+str(len(layers)-1)+''' ; cyc++){
        overall_cycles += cycles[cyc];
        printf("Cycles taken in layer %d: %llu\\n", cyc,cycles[cyc]);
    }
    printf("Overall cycles taken: %llu\\n",overall_cycles);
\n
    return 0;
}\n
''' 

f = open(test_file_name, "w")
f.write(test_content)
f.close()